In [2]:
# Eclipse Viewer — Observer's Perspective
Simulates what an eclipse looks like from any location on Earth.  
Select an eclipse, set your latitude/longitude, and use the **time slider** to watch the Moon cross the Sun.

SyntaxError: invalid syntax (2612845927.py, line 2)

In [8]:
import json
import re
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown

%matplotlib inline

In [9]:
# ============================================================
# Load eclipse data exported by EclipseData.ipynb
# ============================================================
with open("eclipse_data.json") as f:
    data = json.load(f)

eclipse_list = data["eclipse_list"]
print(f"Loaded {len(eclipse_list)} eclipses from: {data['source_url']}")
print(f"Date range: {eclipse_list[0]['date_raw']}  →  {eclipse_list[-1]['date_raw']}")

Loaded 224 eclipses from: https://eclipse.gsfc.nasa.gov/SEcat5/SE2001-2100.html
Date range: 2001 Jun 21  →  2100 Sep 04


In [ ]:
# ============================================================
# GEOMETRY HELPERS
# ============================================================

def parse_coord(coord_str):
    """Parse NASA coordinate string like '11S' or '131W' to signed degrees."""
    if not coord_str or coord_str.strip() == "-":
        return 0.0
    match = re.match(r"(\d+)([NSEW])", coord_str.strip())
    if match:
        val = float(match.group(1))
        if match.group(2) in ("S", "W"):
            val = -val
        return val
    return 0.0


def haversine_km(lat1, lon1, lat2, lon2):
    """Great-circle distance in km between two points on Earth."""
    R = 6371.0
    la1, lo1, la2, lo2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = la2 - la1
    dlon = lo2 - lo1
    a = np.sin(dlat / 2) ** 2 + np.cos(la1) * np.cos(la2) * np.sin(dlon / 2) ** 2
    return R * 2 * np.arcsin(np.sqrt(np.clip(a, 0, 1)))


def overlap_fraction(sun_r, moon_r, d):
    """
    Fraction of the Sun's disk area obscured by the Moon.
    
    sun_r, moon_r : apparent radii (same units)
    d             : center-to-center distance
    """
    if d >= sun_r + moon_r:
        return 0.0
    if d <= abs(sun_r - moon_r):
        return 1.0 if moon_r >= sun_r else (moon_r / sun_r) ** 2

    # Partial overlap — lens-shaped intersection of two circles
    cos1 = np.clip((d**2 + sun_r**2 - moon_r**2) / (2 * d * sun_r), -1, 1)
    cos2 = np.clip((d**2 + moon_r**2 - sun_r**2) / (2 * d * moon_r), -1, 1)
    a1 = sun_r**2 * np.arccos(cos1)
    a2 = moon_r**2 * np.arccos(cos2)
    det = (-d + sun_r + moon_r) * (d + sun_r - moon_r) * \
          (d - sun_r + moon_r) * (d + sun_r + moon_r)
    a3 = 0.5 * np.sqrt(max(det, 0))
    return (a1 + a2 - a3) / (np.pi * sun_r**2)


def get_eclipse_params(eclipse):
    """Unpack useful numbers from an eclipse dict."""
    raw = eclipse.get("_raw", {})
    ecl_lat = parse_coord(raw.get("latitude", "0N"))
    ecl_lon = parse_coord(raw.get("longitude", "0E"))
    magnitude = eclipse.get("magnitude") or 0.95

    pw = raw.get("path_width_km", "-")
    try:
        path_w = float(pw)
    except (ValueError, TypeError):
        path_w = 0.0          # partial eclipses have no path width

    gamma_str = raw.get("gamma", "0")
    try:
        gamma = float(gamma_str)
    except (ValueError, TypeError):
        gamma = 0.0

    return ecl_lat, ecl_lon, magnitude, path_w, gamma


def min_perp_distance_to_path(obs_lat, obs_lon, ecl_lat, ecl_lon, gamma=0.0, max_along_track_km=7500):
    """
    Estimate minimum perpendicular distance from observer to the eclipse
    centerline.  The eclipse path is a LINE across the Earth, not a point.
    
    Uses the cross-track distance formula with a latitude-range gate
    based on gamma (how close the shadow axis passes to Earth's center).
    High |gamma| → path only covers a narrow band of latitudes.
    """
    R = 6371.0
    d13_km = haversine_km(obs_lat, obs_lon, ecl_lat, ecl_lon)

    # If already very close to the greatest eclipse point, skip
    if d13_km < 200:
        return d13_km

    # ── Latitude-range gate ──
    # gamma tells us how far the shadow axis is from Earth's center.
    # High |gamma| → shadow barely grazes the Earth → narrow latitude band.
    # arccos(|gamma|) gives the angular half-range of latitudes the path
    # can cover.  We add a small buffer for safety.
    gamma_abs = min(abs(gamma), 0.999)
    lat_half_range = np.degrees(np.arccos(gamma_abs)) * 0.55 + 5.0
    if abs(obs_lat - ecl_lat) > lat_half_range:
        return d13_km          # observer too far north/south of the path

    d13 = d13_km / R           # angular distance in radians

    # Bearing from eclipse center to observer
    lat1 = np.radians(ecl_lat)
    lon1 = np.radians(ecl_lon)
    lat2 = np.radians(obs_lat)
    lon2 = np.radians(obs_lon)
    dlon = lon2 - lon1
    x = np.sin(dlon) * np.cos(lat2)
    y = (np.cos(lat1) * np.sin(lat2)
         - np.sin(lat1) * np.cos(lat2) * np.cos(dlon))
    brg13 = np.arctan2(x, y)          # bearing to observer (radians)

    min_xt = d13_km                    # fallback = direct distance

    # Try path bearings 0°–178° in 2° steps
    for b_deg in range(0, 180, 2):
        brg12 = np.radians(b_deg)
        xt = abs(R * np.arcsin(
            np.clip(np.sin(d13) * np.sin(brg13 - brg12), -1, 1)
        ))
        if xt < min_xt:
            # Check along-track distance is within half the path length
            cos_xt = np.cos(xt / R)
            if cos_xt > 1e-10:
                cos_at = np.clip(np.cos(d13) / cos_xt, -1, 1)
                at = R * np.arccos(cos_at)
                if at <= max_along_track_km:
                    min_xt = xt

    return min_xt


def viewer_offset(obs_lat, obs_lon, eclipse):
    """
    How far off-center the Moon passes the Sun for this observer.
    Returns offset in Sun-radii units (0 = perfect center).
    
    Model:
    - Estimates perpendicular distance to the eclipse PATH (not just the
      point of greatest eclipse) so that cities along the path of totality
      correctly show a total eclipse.
    - Uses gamma to limit the latitude range (high gamma = narrow path band).
    - Inside the central path width → offset ≈ 0
    - Beyond that → linear ramp to edge of visibility (~3 500 km)
    - Partial eclipses use |gamma| as a base offset
    """
    ecl_lat, ecl_lon, mag, path_w, gamma = get_eclipse_params(eclipse)
    sun_r = 1.0
    moon_r = mag * sun_r
    max_off = sun_r + moon_r          # beyond this = no eclipse

    dist_km = haversine_km(obs_lat, obs_lon, ecl_lat, ecl_lon)
    vis_km = 3500.0                    # rough visibility radius

    if path_w > 0:                     # central eclipse (T, A, H)
        half = path_w / 2.0
        # Use perpendicular distance to estimated path, not point distance
        perp_km = min_perp_distance_to_path(
            obs_lat, obs_lon, ecl_lat, ecl_lon, gamma=gamma
        )
        if perp_km <= half:
            return 0.0
        excess = perp_km - half
        return min(excess / vis_km * max_off, max_off + 0.3)
    else:                              # partial eclipse
        base = abs(gamma) * 0.6
        return min(base + dist_km / vis_km * 0.6, max_off + 0.3)

print("✓ Geometry helpers loaded.")

✓ Geometry helpers loaded.


In [11]:
# ============================================================
# DRAWING FUNCTION — renders the observer's sky view
# ============================================================

def draw_eclipse_sky(eclipse, obs_lat, obs_lon, time_frac):
    """
    Render what the eclipse looks like from (obs_lat, obs_lon).

    time_frac : float in [-1, 1]
        -1 = Moon approaching from the right
         0 = mid-eclipse (closest approach)
        +1 = Moon departing to the left
    """
    ecl_lat, ecl_lon, mag, path_w, _ = get_eclipse_params(eclipse)

    sun_r  = 1.0
    moon_r = mag * sun_r
    offset = viewer_offset(obs_lat, obs_lon, eclipse)

    # Moon travels horizontally across the Sun
    travel = sun_r + moon_r + 0.5
    mx = -time_frac * travel       # right → left
    my = offset

    # Center-to-center distance & obscuration
    d   = np.sqrt(mx**2 + my**2)
    obs_frac = overlap_fraction(sun_r, moon_r, d)

    dist_km = haversine_km(obs_lat, obs_lon, ecl_lat, ecl_lon)
    ecl_type = eclipse.get("type", "?")

    # ---- figure ----
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.set_xlim(-2.5, 2.5)
    ax.set_ylim(-2.5, 2.5)
    ax.set_aspect("equal")

    # Sky color: darken with obscuration
    bright = max(0.02, 0.12 - 0.10 * obs_frac)
    ax.set_facecolor((bright, bright, bright + 0.12))

    # Corona glow — visible when >90 % obscured
    if obs_frac > 0.90:
        alpha_base = (obs_frac - 0.90) / 0.10 * 0.45
        for i in range(10):
            r = sun_r + 0.05 * (i + 1)
            a = alpha_base * (1 - i / 10)
            ax.add_patch(plt.Circle((0, 0), r,
                         color="#FFE4B5", alpha=max(a, 0), lw=0))

    # Sun disk
    ax.add_patch(plt.Circle((0, 0), sun_r, color="#FFD700", zorder=1))

    # Subtle limb-darkening rings
    for i in range(4):
        ax.add_patch(plt.Circle((0, 0), sun_r * (1 - 0.05 * i),
                     fill=False, ec="#FFA500", alpha=0.08, lw=1, zorder=1))

    # Moon disk
    ax.add_patch(plt.Circle((mx, my), moon_r, color="#111111", zorder=2))
    ax.add_patch(plt.Circle((mx, my), moon_r,
                 fill=False, ec="#333333", lw=1.5, zorder=3))

    # Stars when sky is dark enough
    if obs_frac > 0.85:
        rng = np.random.RandomState(42)
        n_stars = int(30 * (obs_frac - 0.85) / 0.15)
        sx = rng.uniform(-2.4, 2.4, n_stars)
        sy = rng.uniform(-2.4, 2.4, n_stars)
        ax.scatter(sx, sy, s=1, color="white", alpha=0.6, zorder=0)

    # Title & info
    ax.set_title(
        f"Eclipse View:  {eclipse['date_raw']}  —  {ecl_type}\n"
        f"Observer: {obs_lat:.1f}°N, {obs_lon:.1f}°E   |   "
        f"{dist_km:,.0f} km from center   |   "
        f"Obscuration: {obs_frac:.1%}",
        color="white", fontsize=11, fontweight="bold", pad=14,
    )
    info = f"Magnitude: {mag}   |   Saros: {eclipse.get('saros','?')}"
    if path_w > 0:
        info += f"   |   Path width: {path_w:.0f} km"
    ax.text(0, -2.35, info, ha="center", color="#aaaaaa", fontsize=9)
    ax.axis("off")
    plt.tight_layout()
    plt.show()

print("✓ Drawing function loaded.")

✓ Drawing function loaded.


In [12]:
# ============================================================
# INTERACTIVE VIEWER
# ============================================================
# • Dropdown — pick any of the 224 eclipses
# • Time slider — animate the Moon crossing the Sun
# • Lat / Lon sliders — move your observer position
#
# Tip: set continuous_update=False so the plot only redraws
#      when you release the slider (much smoother).
# ============================================================

# Build dropdown options
eclipse_options = {}
for i, e in enumerate(eclipse_list):
    label = f"{e['date_raw']}  —  {e['type']}  (Mag: {e['magnitude']})"
    eclipse_options[label] = i

def _update(idx, time_frac, lat, lon):
    draw_eclipse_sky(eclipse_list[idx], lat, lon, time_frac)

interact(
    _update,
    idx=Dropdown(options=eclipse_options, value=0, description="Eclipse:"),
    time_frac=FloatSlider(
        min=-1.0, max=1.0, step=0.02, value=-1.0,
        description="Time:", continuous_update=False,
        style={"description_width": "60px"},
        layout={"width": "500px"},
    ),
    lat=FloatSlider(
        min=-90, max=90, step=0.5, value=0.0,
        description="Lat (°N):", continuous_update=False,
        style={"description_width": "70px"},
        layout={"width": "500px"},
    ),
    lon=FloatSlider(
        min=-180, max=180, step=0.5, value=0.0,
        description="Lon (°E):", continuous_update=False,
        style={"description_width": "70px"},
        layout={"width": "500px"},
    ),
);

interactive(children=(Dropdown(description='Eclipse:', options={'2001 Jun 21  —  Total  (Mag: 1.0495)': 0, '20…